In [ ]:
import rasterio
import rasterio.features
import datacube
import numpy
from datacube.storage import masking
import xarray
import argparse
import os.path
from osgeo import ogr
from osgeo import gdal
from osgeo import osr
import pandas
from datacube.storage.storage import write_dataset_to_netcdf

def xarray_to_cfnetcdf(data_xarray, output_nc_file, variable_name, crs):
    # Data Cube friendly dataset, copy booleans to int8 as bool is not supported
    dcf_ds = data_xarray.astype('int8', copy=False).to_dataset(name = variable_name)
    # set a valid crs object, DC relies upon the python object so a WKT representation of CRS will fail
    dcf_ds.attrs['crs'] = crs
    # Set units for year coordinate
    dcf_ds.coords['time'].attrs['units'] = 'date and time of observations'
    # Set units for data variable    
    dcf_ds.data_vars[variable_name].attrs['units'] = 1
    # write dataset out using datacube storage method - this is an unfortunate nessicity and we should expose a
    # function like this in a nicer way
    write_dataset_to_netcdf(dcf_ds, output_nc_file)

def pq_fuser(dest, src):
    valid_bit = 8
    valid_val = (1 << valid_bit)

    no_data_dest_mask = ~(dest & valid_val).astype(bool)
    numpy.copyto(dest, src, where=no_data_dest_mask)

    both_data_mask = (valid_val & dest & src).astype(bool)
    numpy.copyto(dest, src & dest, where=both_data_mask)

def calcMangFCPVMangPxlFromCube(tileNCFile, tileNCAMCFile, tileAFile, minLat, maxLat, minLon, maxLon, mangShpExt, pv_thresh):

    dc = datacube.Datacube(app='CalcAnnualMangroveExtent')
    
    #Define wavelengths/bands of interest, remove this kwarg to retrieve all bands
    bands_of_interest = ['PV']
    
    #Define sensors of interest
    sensors = ['ls8', 'ls7', 'ls5']
    
    #define temporal range
    start_of_epoch = '1987-01-01'
    # latest observation
    end_of_epoch = '2016-12-31'
    
    query = {'time': (start_of_epoch, end_of_epoch),}
    query['x'] = (minLon, maxLon)
    query['y'] = (maxLat, minLat)
    query['crs'] = 'EPSG:4326'
    
    #Define which pixel quality artefacts you want removed from the results
    mask_components = {'cloud_acca':          'no_cloud',
                       'cloud_shadow_acca' :  'no_cloud_shadow',
                       'cloud_shadow_fmask' : 'no_cloud_shadow',
                       'cloud_fmask' :        'no_cloud',
                       'blue_saturated' :     False,
                       'green_saturated' :    False,
                       'red_saturated' :      False,
                       'nir_saturated' :      False,
                       'swir1_saturated' :    False,
                       'swir2_saturated' :    False,
                       'contiguous' :         True }
    
    print("Read pixel image data into memory.")
    sensor_clean = {}
    for sensor in sensors:
        print(sensor)
        #Load the FC and corresponding PQ
        sensor_fc = dc.load(product=sensor+'_fc_albers', group_by='solar_day', measurements=bands_of_interest, **query)
        if bool(sensor_fc):
            sensor_pq = dc.load(product=sensor+'_pq_albers', group_by='solar_day', fuse_func=pq_fuser, **query)
            # Get the projection info
            crs = sensor_fc.crs
            affine = sensor_fc.affine
            # Apply the PQ masks to the FC
            cloud_free = masking.make_mask(sensor_pq, **mask_components)
            good_data = cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
            sensor_fc = sensor_fc.where(good_data)
            sensor_clean[sensor] = sensor_fc
    
    if bool(sensor_clean):
        print("Merge data from different sensors.")
        fc_clean = xarray.concat(sensor_clean.values(), dim='time')
        time_sorted = fc_clean.time.argsort()
        fc_clean = fc_clean.isel(time=time_sorted)
        
        print("Create Composite")
        annual = fc_clean.resample('A', dim='time', how='median', keep_attrs=True)
        
        print("Rasterise the GMW extent map for the area of interest.")
        # Define pixel size and NoData value of new raster
        xres = affine[0]
        yres = affine[4]
        noDataVal = -9999
        
        # Set the geotransform properties
        xcoord = annual.coords['x'].min()
        ycoord = annual.coords['y'].max()
        geotransform = (xcoord - (xres*0.5), xres, 0, ycoord + (yres*0.5), 0, yres)
        
        # Open the data source and read in the extent
        source_ds = ogr.Open(mangShpExt)
        source_layer = source_ds.GetLayer()
        source_srs = source_layer.GetSpatialRef()
        vx_min, vx_max, vy_min, vy_max = source_layer.GetExtent() # This is extent of Australia
        
        # Create the destination extent
        yt,xt = annual.PV.isel(time=1).shape
        
        # Set up 'in-memory' gdal image to rasterise the shapefile too
        target_ds = gdal.GetDriverByName('MEM').Create('', xt, yt, gdal.GDT_Byte)
        target_ds.SetGeoTransform(geotransform)
        albers = osr.SpatialReference()
        albers.ImportFromEPSG(3577)
        target_ds.SetProjection(albers.ExportToWkt())
        band = target_ds.GetRasterBand(1)
        band.SetNoDataValue(noDataVal)
        
        # Rasterise
        gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[1])
        
        # Read as array the GMW mask
        gmwMaskArr = band.ReadAsArray()
        
        
        print("Apply the GMW Mask to the FCPV values")
        mangroveannual = (annual.PV.where(gmwMaskArr == 1))
        
        print("Apply thresholds to FCPV to find total mangrove mask.")
        mangroveAreaPxlC = mangroveannual>pv_thresh
        
        print("Calculate the number of pixels within the mangrove mask and write to CSV file.")
        numMangPxls = numpy.sum(mangroveAreaPxlC.data)

        for datetimeVal in annual.time:
            yearVal=datetimeVal['time.year'].data

            numMangPxls = numpy.sum(mangroveAreaPxlC.sel(time=datetimeVal).data)

            pxlCountSeries = pandas.Series([numMangPxls], index=['MangPxls'])
            tileAFileOut = tileAFile+'_'+str(yearVal)+'.csv'
            pxlCountSeries.to_csv(tileAFileOut)
            
            tileNCAMCFileOut = tileNCAMCFile+'_'+str(yearVal)+'.nc'
            tileNCFileOut = tileNCFile+'_'+str(yearVal)+'.nc'

            xarray_to_cfnetcdf(mangroveAreaPxlC.sel(time=datetimeVal), tileNCAMCFileOut, 'PV'+str(yearVal), crs)           
            xarray_to_cfnetcdf(mangroveannual.sel(time=datetimeVal), tileNCFileOut, 'PV'+str(yearVal), crs)         

In [ ]:
working_dir =  os.path.expanduser('~/mangrove/')

#output files
tileNCFile = working_dir + 'pv_1695_14115_0_pv'
tileNCAMCFile = working_dir + 'pv_1695_14115_0_mangmask'
tileAFile = working_dir + 'pv_1695_14115_0_pxlcounts'
#This notebook expects you to have access to this file
gmwMangExtShp = working_dir + 'GMW_Australia_MangroveExtent2010_AlbersEA_shp.shp'
minlat = -17.0 
maxlat = -16.9 
minlon = 141.1
maxlon = 141.2
pv_thresh = 30
calcMangFCPVMangPxlFromCube(tileNCFile, tileNCAMCFile, tileAFile, minlat, maxlat, minlon, maxlon, gmwMangExtShp, pv_thresh)